<a href="https://colab.research.google.com/github/cleftwitch28/SOCL-PYQT5/blob/main/ANN_SOCL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

# Define a simple ANN architecture
class ANN(nn.Module):
    def __init__(self):
        super(ANN, self).__init__()
        self.fc1 = nn.Linear(3 * 32 * 32, 128)  # Assuming 3-channel (RGB) images of size 32x32
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 4)  # 10 classes in CIFAR-10

    def forward(self, x):
        x = x.view(-1, 3 * 32 * 32)  # Flatten the input
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.Resize(32),  # Resize images to 32x32
    transforms.CenterCrop(32),  # Center crop to ensure 32x32
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
trainset = torchvision.datasets.ImageFolder(
        root="/content/drive/MyDrive/Dataset/Train",
        transform=transform
    )
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Initialize the model and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = ANN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# Training loop
for epoch in range(10):  # Adjust the number of epochs as needed
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if i % 100 == 99:
            print(f"Epoch {epoch+1}, Batch {i+1}, Loss: {loss.item():.4f}")

print("Training finished!")

# Now you can use the trained model for inference on new images
# (Remember to preprocess the input image similarly to the training data)


In [ ]:
y_test="/content/drive/MyDrive/Dataset/test"

test_dataset = torchvision.datasets.ImageFolder(
        root=y_test,
        transform=transform
    )
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True)
true_labels = []
predictions = []
with torch.no_grad(): # No need to calculate gradients for evaluation
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        true_labels.extend(labels.cpu().numpy())
        predictions.extend(predicted.cpu().numpy())

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
# Assuming 'y_test' contains ground truth labels and 'predictions' contains model predictions
cm = confusion_matrix(true_labels, predictions)
labels = ["Alluvial Soil","Black Soil","Clay Soil","Red Soil"]

df_cm = pd.DataFrame(cm, index = labels, columns = labels)
plt.figure(figsize = (7,7))
sns.heatmap(df_cm, annot=True, cmap=plt.cm.Blues, fmt='g')
plt.xlabel("Predicted Shape", fontsize = 20)
plt.ylabel("True Shape", fontsize = 20)
plt.show()


In [ ]:
from sklearn.metrics import roc_curve, auc, f1_score

fpr, tpr, thresholds = roc_curve(true_labels, predictions)
roc_auc = auc(fpr, tpr)

In [ ]:
def Predict_class():
  inp="/content/drive/MyDrive/Dataset/test/Clay soil/Clay_4.jpg"
  from PIL import Image # Import Image from PIL to open image file
  img = Image.open(inp)
  transform = transforms.Compose([
    transforms.Resize(32),  # Resize images to 32x32
    transforms.CenterCrop(32),  # Center crop to ensure 32x32
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
  inp = transform(img)
  output=net(inp)
  print(output.argmax())
  if output.argmax() == 0:
      print("The given Image belongs to Alluvial Soil")
  elif output.argmax() == 1:
    print("The given Image belongs to Black Soil")
  elif output.argmax() == 2:
    print("The given Image belongs to Clay Soil")
  elif output.argmax() == 3:
    print("The given Image belongs to Red Soil")

Predict_class()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')